#### Author : Saurabh & Chee Weng
#### Objective : 
1. To enable users to generate output for books description in the style that users want. 
2. Being able to try Gemini, Gemini with Grounding, and Anthropic latest models too.
3. Although not shown but this approach can be extended to llama3.1 and other models. 

In [1]:
# !pip install google-cloud-aiplatform

In [2]:
import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}
! mkdir output

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

Updated property [core/project].
Creating gs://my-project-0004-346516-pytorch112kagglewbi-asia-southeast1/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.
mkdir: cannot create directory ‘output’: File exists


### Generation just using Gemini (without grounding) 

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerationConfig, GenerativeModel

vertexai.init(project=PROJECT_ID, location=LOCATION)

model = GenerativeModel("gemini-1.5-pro-001")



### Generation just using Gemini with Grounding

In [10]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
    grounding,
)

# TODO (developer): update project_id
vertexai.init(project=PROJECT_ID, location=LOCATION)

model = GenerativeModel("gemini-1.5-pro-001")

# Use Google Search for grounding
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

# prompt = "When is the next total solar eclipse in US?"
# Example usage
title = "Killer Croc hunter / written by Scott Sonneborn ; illustrated by Mike DeCarlo, Erik Doescher, and Lee Loughridge ; Batman created by Bob Kane."
author = "Scott Sonneborn"
description = "As a pro wrestler, Killer Croc bullied his opponents inside the ring. As a cutthroat criminal, this ferocious freak forces his competition out of town. Next on his hoodlum hit list is Kite Man, but the second-rate super-villain isn't leaving Gotham without a fight. He's partnering with Batman, the world's greatest detective, to wrangle up the reptilian rogue and take him down for the count"
language = "Chinese"

prompt = f"""
    Given the following book information:
    Title: {title}
    Author: {author}
    Description: {description}

    Generate an engaging call-to-action text to promote this book. The text should be concise, exciting, and encourage readers to check out the book.
    
    produce output in {language} only
    """
    
response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
    ),
)

print(response.text)

准备好迎接一场狂野的冒险吧！《杀手鳄猎人》现已震撼登场！当哥谭市的残酷恶棍杀手鳄将目光投向风筝人时，一场大战即将爆发。蝙蝠侠能否战胜这个爬行动物的暴行，还是杀手鳄会把哥谭变成他的狩猎场？立即抢购，见证这场史诗般的对决！ 



### Use Clause model
##### Do enable clause model by filling up form from Model Garden first. 

In [11]:
from anthropic import AnthropicVertex

LOCATION="europe-west1" # or "us-east5"

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

message = client.messages.create(
  max_tokens=1024,
  messages=[
    {
      "role": "user",
      "content": prompt,
    }
  ],
  model="claude-3-5-sonnet@20240620",
)
print(message.model_dump_json(indent=2))

{
  "id": "msg_vrtx_013MmwYURL7NEkiGMk7w6bag",
  "content": [
    {
      "text": "哥特市的夜晚将再次沸腾！蝙蝠侠与罪犯联手对抗凶残的杀手鳄鱼！这本精彩刺激的漫画书将带你进入一个充满动作和悬疑的世界。Scott Sonneborn的精彩文字搭配Mike DeCarlo、Erik Doescher和Lee Loughridge的精美插图，为你呈现一场惊心动魄的冒险。你准备好加入这场哥特市的地下之战了吗？立即获取《杀手鳄鱼猎人》，体验蝙蝠侠的智慧与勇气如何与意想不到的盟友携手制服这个可怕的对手！不要错过这本令人兴奋的漫画大作，现在就行动吧！",
      "type": "text"
    }
  ],
  "model": "claude-3-5-sonnet-20240620",
  "role": "assistant",
  "stop_reason": "end_turn",
  "stop_sequence": null,
  "type": "message",
  "usage": {
    "input_tokens": 220,
    "output_tokens": 248
  }
}


In [12]:
# call_to_action = generate_response_with_anthropic(title, author, description)

In [13]:
# print(call_to_action)

## Prompt enhancement V3

In [15]:
import csv
import vertexai
from vertexai.preview.generative_models import GenerationConfig, GenerativeModel

vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel("gemini-1.5-pro-002")

In [33]:
few_shot_examples =  """
INPUT Title: The Martian

INPUT Author: Andy Weir

INPUT Description: An astronaut becomes stranded on Mars after his team assumes him dead, and must rely on his ingenuity to find a way to signal to Earth that he is alive.   

OUTPUT AI-generated Call-to-Action: Stranded on Mars, one man's fight for survival begins. Can he defy the odds and make it home? Experience the thrilling adventure of The Martian today!

Example 2

INPUT Title: Pride and Prejudice

INPUT Author: Jane Austen

INPUT Description: Follow the spirited Elizabeth Bennet as she navigates the complexities of love and societal expectations in 19th-century England. Will she find happiness amidst the prejudices and pride of her world?   

OUTPUT AI-generated Call-to-Action: Wit, romance, and social satire intertwine in this timeless classic. Discover the enduring charm of Pride and Prejudice - a must-read for every generation!

Example 3

INPUT Title: The Hunger Games

INPUT Author: Suzanne Collins

INPUT Author: In a dystopian future, Katniss Everdeen volunteers to take her younger sister's place in the Hunger Games, a televised event in which teenagers must fight to the death.

OUTPUT AI-generated Call-to-Action: Enter the arena! Join Katniss in a fight for survival and rebellion against a tyrannical regime. The Hunger Games - an unforgettable story of courage and defiance.

"""

In [38]:
# generate_call_to_action_withGrounding_fewshot

def generate_call_to_action_fewshot_v3(title, author, language, description, few_shot_examples):
    
    prompt_few_shots_v3 = f"""Craft a compelling one-liner to entice readers aged 17+ to borrow this book. Use the following metadata,
    ensuring the language is engaging and suitable for young adults.  
    Create the output with no special characters or markup.
    
    Title: {title}
    Author: {author}
    Language: {language}
    Description: {description}
    
    if Description is empty just use the Title
    
    Few-Shot Examples for Generating Book Call-to-Actions : {few_shot_examples}
    
    Important: If the book's language is specified, craft the one-liner in {language} only 
    """
    
    response = model.generate_content(
        prompt_few_shots_v3,
        generation_config=GenerationConfig(
            temperature=0.1,
            max_output_tokens=50,
        ),
    )

    return response.text

In [39]:
# # Read CSV and process each row
# with open("input_v3.csv", "r") as file:
#     reader = csv.reader(file)
#     next(reader)  # Skip header row (if present)

#     for row in reader:
#         uuid, title, author, language, description, old_call_to_action = row  # Assuming columns are in this order
#         call_to_action = generate_call_to_action_fewshot_v3(title, author, language, description, few_shot_examples)
#         print(f"Call to action for '{title}': {call_to_action}")

In [41]:
import csv

# Read CSV and process each row
with open("input_v3.csv", "r") as file:
    reader = csv.reader(file)
    header = next(reader)  # Store the header row
    rows = list(reader)  # Read all remaining rows into a list

# Generate call-to-actions and update rows
for row in rows:
    uuid, title, author, language, description, old_call_to_action = row  # Assuming columns are in this order
    
    print(f"input data '{uuid} , {title} , {author} , {language}': {description}")
    
    call_to_action_withgemini_v3 = generate_call_to_action_fewshot_v3(title, author, language, description,few_shot_examples)
    print(f"Call to action for '{title} , {language}': {call_to_action_withgemini_v3}")
    
    row.extend([call_to_action_withgemini_v3])
    
    del uuid, title, author, language, description, old_call_to_action, call_to_action_withgemini_v3 

# Write the updated data back to the CSV
with open("output_fewshot_v3.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(header + ["Call to Action with Gemini_fewshot_v3"])
    writer.writerows(rows)



input data 'a15b9438-7eea-45d6-a448-0bb3bb20ece9 , Fitter food / by Sarah Brewer.  , Brewer, Sarah. , English': Brewer combines a balanced approach to food with an appreciation of the fact that busy men need eating that is easy, appetising and healthy. She fills gaps in nutritional knowledge, and gives programmes for putting on and losing weight. 
Call to action for 'Fitter food / by Sarah Brewer.  , English': Fuel your body and crush your goals with Fitter Food's easy, healthy, and delicious recipes.

input data '5d2323b2-fc35-4ff1-8cb5-d58bc0e96790 , Fantastic voyage : live long enough to live forever / Ray Kurzweil, Terry Grossman.  , Kurzweil, Ray. , English': 
Call to action for 'Fantastic voyage : live long enough to live forever / Ray Kurzweil, Terry Grossman.  , English': Hack your biology and journey towards immortality in Fantastic Voyage: Live Long Enough to Live Forever.

input data '541867bc-6984-4603-837f-72a14ffba0fb , Easy nickels and other stories on humour.  ,  , Engl